In [1]:
import numpy as np
import requests
import pandas as pd
from my_utils import *
from tqdm import tqdm
tqdm.pandas()

In [2]:
api_key = 'RGAPI-d12d74b8-e085-4a99-b167-f7196531ea67'

In [3]:
url = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page=1&api_key='+api_key

In [4]:
res = requests.get(url).json()

In [5]:
summonerName_lst = list(map(lambda x: x['summonerName'], res))

In [6]:
summonerName_lst

['Sehee lu',
 'Jasm1ne',
 '꿀태영',
 '탈론등장',
 '싱거워',
 '갓병대 욱',
 '카리Na',
 'T1 Keria fans',
 '김홍순3',
 '지겨워죽겠엉',
 '벡스 갈리오',
 '종토리',
 '공새롬 유튜브',
 'SEK4',
 '신미야',
 '22 05 29bb',
 '메일혐오자',
 'Fatty',
 'mid revenant',
 '법무법인 김앤장',
 '코오즈네 지리',
 '얼음 연꽃',
 'F1F2F3F4DF',
 'SUPPOSSED',
 'Lovely',
 '혁둥잉',
 'SN Bin',
 '짜피나크면못막아',
 '띠동갑남친',
 '슬프면우서',
 '올때 거북R',
 '공 볼',
 '서른웅',
 'Love liked',
 '월요일이시른사람',
 'Youtube Axiyo',
 '상 상',
 '돌망치 오우거',
 'TFT잘하는사람',
 '탑 라인전 최약체',
 '패배가무서운사람',
 '바이블린',
 'lihai1',
 '도구 교과서',
 '샤코0',
 '이성하',
 'IGOLNIK',
 '레넥톤따먹고싶당',
 '매너플레이합죠',
 '트위치 카제',
 'moochin련',
 '해뤼이 케케인',
 'Stay Sunday',
 'Taric R',
 '건쵸비',
 '평발이',
 '유창하다',
 'Migung',
 '허허멘',
 '나비꽃가루빼앗기',
 '카운트다운4',
 'itsuka Kotor1',
 '덤벼봐 죽어줄게',
 '싸우지들마세여',
 '엑쓰맨',
 '첼린저갈피지컬',
 '어둠잔fan',
 '나 마두팔이오',
 '카서스뽀삐쉔',
 '말상김민성',
 '고양고양이 고양',
 '금발여신태연',
 '기백짱짱123',
 'skyplay',
 '어른도치씩씩',
 'llllIIllllllIIII',
 '순수하던',
 'alalalalalala',
 'Imagine that',
 '에 디',
 '아 뭔 미드근코여',
 '도 빈',
 'LoL Catarina',
 '창 의',
 '지 띵',
 '포뇨의지옥참마도',
 'AttxcK',

In [9]:
def get_lst(summonerName_lst):
    tmp = []
    for i in summonerName_lst[:3]:
        try:
            puuid = get_puuid(i)
            matchid = get_matchid(puuid, 3)
            matchs_timelines = get_matchs_timelines(matchid)
            tmp.extend(matchs_timelines)
        except:
            continue
    return tmp

In [10]:
tmp = get_lst(summonerName_lst)

In [11]:
df = pd.DataFrame(tmp, columns = ['gameId', 'matchs', 'timelines'])

In [12]:
df

,gameId,matchs,timelines
0,KR_5959757493,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_5959723670,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_5959674785,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_6026195421,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_6026183033,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
5,KR_6026067992,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [27]:
df.iloc[0].timelines['info']['frames'][0]['participantFrames']['1']['totalGold']

500

In [60]:
def assist_calc(x): 
    try:
        return '.'.join(list(map(lambda y: str(y), x['assistingParticipantIds'])))
    except:
        return '-'

In [222]:
tmp_df = df.copy()
lst = []
for i in range(len(tmp_df)):
    g_id = tmp_df.iloc[i].gameid
    match = tmp_df.iloc[i].matchs['info']
    timeline = tmp_df.iloc[i].timelines['info']
    
    tmp = list(map(lambda x: x['events'], timeline['frames']))
    event_lst = [element for array in tmp for element in array]
    
    for j in range(10):
        lst_tmp = []
        lst_tmp.append(str(g_id))
        lst_tmp.append(match['gameDuration'])
        lst_tmp.append(match['gameVersion'])
        lst_tmp.append(match['participants'][j]['summonerName'])
        lst_tmp.append(match['participants'][j]['participantId'])
        lst_tmp.append(match['participants'][j]['championName'])
        lst_tmp.append(match['participants'][j]['teamPosition'])
        lst_tmp.append(match['participants'][j]['teamId'])
        lst_tmp.append(match['participants'][j]['win'])
        lst_tmp.append(match['participants'][j]['kills'])
        lst_tmp.append(match['participants'][j]['deaths'])
        lst_tmp.append(match['participants'][j]['assists'])
        lst_tmp.append(match['participants'][j]['totalDamageDealtToChampions'])
        lst_tmp.append(match['participants'][j]['totalDamageTaken'])
        str_tmp = ''
        for k in range(len(timeline['frames'])):
            try:
                str_tmp += (str(timeline['frames'][k]['participantFrames'][str(j)]['totalGold'])+'|')
            except:
                str_tmp += ''
        lst_tmp.append(str_tmp)
        
        for t in range(len(event_lst)):
            if event_lst[t]['type'] == 'BUILDING_KILL':
                if match['participants'][j]['teamId'] == event_lst[t]['teamId']:
                    if match['participants'][j]['teamPosition'][:3] == event_lst[t]['laneType'][:3]:
                        timestamp = event_lst[t]['timestamp']
                        try:
                            firstDT = event_lst[t]['firstDT']
                        except:
                            firstDT = 'none'
        lst_tmp.append(timestamp)
        lst_tmp.append(firstDT)
                
        ban_blue = list(map(lambda x: str(x['championId']), match['teams'][0]['bans']))
        ban_red = list(map(lambda x: str(x['championId']), match['teams'][1]['bans']))
        ban_lst_tmp = list(set(ban_blue + ban_red))
        ban_lst = '|'.join(ban_lst_tmp)
        lst_tmp.append(ban_lst)
        kill_log = list(map(lambda x: list(filter(lambda y: y['type'] == 'CHAMPION_KILL', x['events'])), timeline['frames']))
        kill_log = [element for array in kill_log for element in array]
        k = list(map(lambda x: str(x['killerId']), kill_log))
        v = list(map(lambda x: str(x['victimId']), kill_log))
        a = list(map(assist_calc, kill_log))
        lst_tmp.append('|'.join(k))
        lst_tmp.append('|'.join(v))
        lst_tmp.append('|'.join(a))
        
        lst.append(lst_tmp)

In [223]:
lst

[['KR_5959757493',
  1689,
  '12.11.445.6412',
  'olaolaolaola',
  1,
  'Graves',
  'TOP',
  100,
  False,
  0,
  6,
  1,
  6977,
  19602,
  '',
  1495974,
  'none',
  '221|-1|58|7|41|55|62|53',
  '6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|5|3|8|7|10|7|8|8|8|8|6|4|8|4|2|9|7|8|7|8',
  '3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|10|7|4|6|9|5|2|1|3|1|2|3|2|1|10|5|8|10|1|4|5|3|2',
  '-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|-|4.5|4.5|5|1.3.5|6|2.3|5|-|10|7.8|8.10|-|10|7|7.10|-|5|7.10|2.5|-|8|6.9|7.9|8|6'],
 ['KR_5959757493',
  1689,
  '12.11.445.6412',
  'Sehee lu',
  2,
  'Nidalee',
  'JUNGLE',
  100,
  False,
  8,
  7,
  2,
  17258,
  29708,
  '500|500|563|860|1204|1599|1984|2267|2654|3072|3401|3756|4160|4616|5062|5506|5784|5958|6312|6872|7238|7638|7781|7903|8258|8680|8802|9252|9541|9559|',
  1495974,
  'none',
  '221|-1|58|7|41|55|62|53',
  '6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|5|3|8|7|10|7|8|8|8|8|6|4|8|4|2|9|7|8|7|8',
  '3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10

In [224]:
match_lst = pd.DataFrame(lst, columns = ['gameid', 'gameDuration', 'gameVersion', 
                                         'summonerName', 'participantId', 
                                         'championName', 'teamPosition', 'teamId', 
                                         'win', 'kills', 'deaths', 'assists', 
                                         'totalDamageDealtToChampions', 
                                         'totalDamageTaken', 'Gold', 'timestamp', 'firstDT', 
                                         'ban', 'k_log', 'v_log', 'a_log'])

In [225]:
match_lst

,gameid,gameDuration,gameVersion,summonerName,participantId,championName,teamPosition,teamId,win,kills,...,assists,totalDamageDealtToChampions,totalDamageTaken,Gold,timestamp,firstDT,ban,k_log,v_log,a_log
0,KR_5959757493,1689,12.11.445.6412,olaolaolaola,1,Graves,TOP,100,False,0,...,1,6977,19602,,1495974,none,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...
1,KR_5959757493,1689,12.11.445.6412,Sehee lu,2,Nidalee,JUNGLE,100,False,8,...,2,17258,29708,500|500|563|860|1204|1599|1984|2267|2654|3072|...,1495974,none,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...
2,KR_5959757493,1689,12.11.445.6412,길고양이츄르뺏기,3,Yone,MIDDLE,100,False,3,...,4,17145,20452,500|500|666|1013|1313|1815|2349|2721|3058|3709...,1516556,none,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...
3,KR_5959757493,1689,12.11.445.6412,Senaphine,4,Seraphine,BOTTOM,100,False,3,...,7,11252,13552,500|500|584|850|1092|1298|1755|2059|2348|2760|...,1415902,none,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...
4,KR_5959757493,1689,12.11.445.6412,need a win,5,Senna,UTILITY,100,False,4,...,10,12510,13125,500|500|563|790|1018|1467|1667|2044|2452|2687|...,1415902,none,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,KR_6005739151,1078,12.12.450.4196,수영장파티조민혁,6,Seraphine,,200,True,8,...,34,26373,18428,1400|1403|1851|2526|3689|4172|4586|5607|6219|6...,1256073,none,,7|5|9|3|9|8|2|5|5|5|7|8|7|7|7|1|7|9|6|10|10|9|...,3|8|5|6|4|3|8|7|9|10|2|5|1|4|3|8|3|4|2|5|1|3|7...,6.8.9.10|1.2.4|6.7.10|-|6|6.7.9.10|1.3.5|1.3.4...
86,KR_6005739151,1078,12.12.450.4196,과단성,7,Yone,,200,True,13,...,19,25933,22133,1400|1403|1891|2481|3233|3709|4566|5037|5747|6...,1256073,none,,7|5|9|3|9|8|2|5|5|5|7|8|7|7|7|1|7|9|6|10|10|9|...,3|8|5|6|4|3|8|7|9|10|2|5|1|4|3|8|3|4|2|5|1|3|7...,6.8.9.10|1.2.4|6.7.10|-|6|6.7.9.10|1.3.5|1.3.4...
87,KR_6005739151,1078,12.12.450.4196,탈북마스터,8,TwistedFate,,200,True,7,...,13,13775,17994,1400|1403|2231|2947|4624|5133|5941|6485|7311|7...,1256073,none,,7|5|9|3|9|8|2|5|5|5|7|8|7|7|7|1|7|9|6|10|10|9|...,3|8|5|6|4|3|8|7|9|10|2|5|1|4|3|8|3|4|2|5|1|3|7...,6.8.9.10|1.2.4|6.7.10|-|6|6.7.9.10|1.3.5|1.3.4...
88,KR_6005739151,1078,12.12.450.4196,영수퍼맨,9,Nidalee,,200,True,11,...,24,29685,23858,1400|1403|1898|2500|3517|4068|4500|5002|5770|6...,1256073,none,,7|5|9|3|9|8|2|5|5|5|7|8|7|7|7|1|7|9|6|10|10|9|...,3|8|5|6|4|3|8|7|9|10|2|5|1|4|3|8|3|4|2|5|1|3|7...,6.8.9.10|1.2.4|6.7.10|-|6|6.7.9.10|1.3.5|1.3.4...


In [19]:
df.iloc[0].timelines['info'].keys()

dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

In [73]:
df.iloc[0].timelines['info']['frames'][0]['participantFrames']

{'1': {'championStats': {'abilityHaste': 0,
   'abilityPower': 0,
   'armor': 33,
   'armorPen': 0,
   'armorPenPercent': 0,
   'attackDamage': 25,
   'attackSpeed': 100,
   'bonusArmorPenPercent': 0,
   'bonusMagicPenPercent': 0,
   'ccReduction': 0,
   'cooldownReduction': 0,
   'health': 625,
   'healthMax': 625,
   'healthRegen': 0,
   'lifesteal': 0,
   'magicPen': 0,
   'magicPenPercent': 0,
   'magicResist': 32,
   'movementSpeed': 340,
   'omnivamp': 0,
   'physicalVamp': 0,
   'power': 325,
   'powerMax': 325,
   'powerRegen': 0,
   'spellVamp': 0},
  'currentGold': 500,
  'damageStats': {'magicDamageDone': 0,
   'magicDamageDoneToChampions': 0,
   'magicDamageTaken': 0,
   'physicalDamageDone': 0,
   'physicalDamageDoneToChampions': 0,
   'physicalDamageTaken': 0,
   'totalDamageDone': 0,
   'totalDamageDoneToChampions': 0,
   'totalDamageTaken': 0,
   'trueDamageDone': 0,
   'trueDamageDoneToChampions': 0,
   'trueDamageTaken': 0},
  'goldPerSecond': 0,
  'jungleMinionsKille

In [10]:
tmp = list(map(lambda x: x['events'], df.iloc[0].timelines['info']['frames']))

In [11]:
event_lst = [element for array in tmp for element in array]

In [12]:
event_lst

[{'realTimestamp': 1654773145487, 'timestamp': 0, 'type': 'PAUSE_END'},
 {'itemId': 1055,
  'participantId': 6,
  'timestamp': 4724,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 6,
  'timestamp': 5153,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 6,
  'timestamp': 5318,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1054,
  'participantId': 3,
  'timestamp': 5550,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1036,
  'participantId': 1,
  'timestamp': 5682,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 3,
  'timestamp': 6012,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3850,
  'participantId': 10,
  'timestamp': 6078,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2031,
  'participantId': 1,
  'timestamp': 6111,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 3,
  'timestamp': 6177,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 1,
  'timestamp': 6507,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'partic

'BUILDING_KILL'

In [82]:
event_lst[0]['type'] == 'BUILDING_KILL'

False

In [115]:
tower_lst = []
for i in range(len(event_lst)):
    if event_lst[i]['type'] == 'BUILDING_KILL':
        tmp_lst = []
        tmp_lst.append(event_lst[i]['laneType'])
        tmp_lst.append(event_lst[i]['teamId'])
        tmp_lst.append(event_lst[i]['timestamp'])
        try:
            tmp_lst.append(event_lst[i]['firstDT'])
        except:
            tmp_lst.append('none')
        tower_lst.append(tmp_lst)

In [116]:
tower_lst

[['TOP_LANE', 100, 838583, 'none'],
 ['MID_LANE', 100, 1004048, 'none'],
 ['BOT_LANE', 100, 1096123, 'none'],
 ['MID_LANE', 200, 1235403, 'none'],
 ['TOP_LANE', 200, 1406358, 'none'],
 ['BOT_LANE', 100, 1415902, 'none'],
 ['TOP_LANE', 100, 1495974, 'none'],
 ['MID_LANE', 100, 1516556, 'none']]

In [117]:
tower = pd.DataFrame(tower_lst, columns = ['laneType', 'teamId', 'timestamp', 'firstDT'])

In [118]:
tower

,laneType,teamId,timestamp,firstDT
0,TOP_LANE,100,838583,none
1,MID_LANE,100,1004048,none
2,BOT_LANE,100,1096123,none
3,MID_LANE,200,1235403,none
4,TOP_LANE,200,1406358,none
5,BOT_LANE,100,1415902,none
6,TOP_LANE,100,1495974,none
7,MID_LANE,100,1516556,none


In [160]:
tmp_df = df.copy()
tower_lst = []
for i in range(len(tmp_df)):
    g_id = tmp_df.iloc[i].gameid
    timeline = df.iloc[i].timelines['info']['frames']
    tmp = list(map(lambda x: x['events'], timeline))
    event_lst = [element for array in tmp for element in array]
    
    for i in range(len(event_lst)):
        if event_lst[i]['type'] == 'BUILDING_KILL':
            tmp_lst = []
            tmp_lst.append(g_id)
            tmp_lst.append(event_lst[i]['laneType'][:3])
            tmp_lst.append(event_lst[i]['teamId'])
            tmp_lst.append(event_lst[i]['timestamp'])
            try:
                tmp_lst.append(event_lst[i]['firstDT'])
            except:
                tmp_lst.append('none')
            tower_lst.append(tmp_lst)

In [161]:
tower = pd.DataFrame(tower_lst, columns = ['gameid', 'laneType', 'teamId', 'timestamp', 'firstDT'])

In [164]:
tower

,gameid,laneType,teamId,timestamp,firstDT,teamPosition
0,KR_5959757493,TOP,100,838583,none,TOP
1,KR_5959757493,MID,100,1004048,none,MIDDLE
2,KR_5959757493,BOT,100,1096123,none,BOT
3,KR_5959757493,MID,200,1235403,none,MIDDLE
4,KR_5959757493,TOP,200,1406358,none,TOP
...,...,...,...,...,...,...
77,KR_6005739151,MID,100,696241,none,MIDDLE
78,KR_6005739151,MID,200,876093,none,MIDDLE
79,KR_6005739151,MID,200,883069,none,MIDDLE
80,KR_6005739151,MID,100,928238,none,MIDDLE


In [163]:
tower['teamPosition'] = tower['laneType'].apply(lambda x: x.replace('MID', 'MIDDLE'))

In [169]:
merge_tmp = pd.merge(match_lst, tower)

In [170]:
merge_tmp

,gameid,gameDuration,gameVersion,summonerName,participantId,championName,teamPosition,teamId,win,kills,...,totalDamageDealtToChampions,totalDamageTaken,Gold,ban,k_log,v_log,a_log,laneType,timestamp,firstDT
0,KR_5959757493,1689,12.11.445.6412,olaolaolaola,1,Graves,TOP,100,False,0,...,6977,19602,,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,TOP,838583,none
1,KR_5959757493,1689,12.11.445.6412,olaolaolaola,1,Graves,TOP,100,False,0,...,6977,19602,,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,TOP,1495974,none
2,KR_5959757493,1689,12.11.445.6412,길고양이츄르뺏기,3,Yone,MIDDLE,100,False,3,...,17145,20452,500|500|666|1013|1313|1815|2349|2721|3058|3709...,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,MID,1004048,none
3,KR_5959757493,1689,12.11.445.6412,길고양이츄르뺏기,3,Yone,MIDDLE,100,False,3,...,17145,20452,500|500|666|1013|1313|1815|2349|2721|3058|3709...,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,MID,1516556,none
4,KR_5959757493,1689,12.11.445.6412,카시스 페델리안,6,Sett,MIDDLE,200,True,3,...,15097,23310,500|500|583|793|1085|1426|1625|1898|2160|2396|...,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,MID,1235403,none
5,KR_5959757493,1689,12.11.445.6412,생배낮듀혐오자,8,Akshan,TOP,200,True,12,...,24412,15480,500|500|606|1023|1378|1500|2098|2765|2987|3548...,221|-1|58|7|41|55|62|53,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,TOP,1406358,none
6,KR_5959723670,1212,12.11.445.6412,qwr245,1,Kennen,TOP,100,False,1,...,13539,14123,,517|-1|7|235|96|234|203|236|68|53,7|10|4|3|4|9|2|6|4|8|9|2|9|10|4|9|6|4|8|8|8|7|...,5|4|9|10|9|3|6|2|7|4|5|7|5|2|7|4|1|9|2|5|4|1|6...,9.10|7.9|5|2|5|7.10|1|-|5|7.9.10|7.8.10|3|7.10...,TOP,1188059,none
7,KR_5959723670,1212,12.11.445.6412,qwr245,1,Kennen,TOP,100,False,1,...,13539,14123,,517|-1|7|235|96|234|203|236|68|53,7|10|4|3|4|9|2|6|4|8|9|2|9|10|4|9|6|4|8|8|8|7|...,5|4|9|10|9|3|6|2|7|4|5|7|5|2|7|4|1|9|2|5|4|1|6...,9.10|7.9|5|2|5|7.10|1|-|5|7.9.10|7.8.10|3|7.10...,TOP,1205338,none
8,KR_5959723670,1212,12.11.445.6412,DamienRice,3,Azir,MIDDLE,100,False,1,...,5864,9004,500|500|621|861|1251|1504|1962|2315|2532|2855|...,517|-1|7|235|96|234|203|236|68|53,7|10|4|3|4|9|2|6|4|8|9|2|9|10|4|9|6|4|8|8|8|7|...,5|4|9|10|9|3|6|2|7|4|5|7|5|2|7|4|1|9|2|5|4|1|6...,9.10|7.9|5|2|5|7.10|1|-|5|7.9.10|7.8.10|3|7.10...,MID,1187167,none
9,KR_5959723670,1212,12.11.445.6412,LianHua Sword,6,Akali,TOP,200,True,6,...,11308,14336,500|500|523|759|894|1133|1449|1646|2081|2215|2...,517|-1|7|235|96|234|203|236|68|53,7|10|4|3|4|9|2|6|4|8|9|2|9|10|4|9|6|4|8|8|8|7|...,5|4|9|10|9|3|6|2|7|4|5|7|5|2|7|4|1|9|2|5|4|1|6...,9.10|7.9|5|2|5|7.10|1|-|5|7.9.10|7.8.10|3|7.10...,TOP,1046081,none


-------------------------------------------------------------------------------------

In [13]:
tower_log = list(filter(lambda x: x['type'] == 'BUILDING_KILL', event_lst))

In [14]:
tower_log

[{'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 8,
  'laneType': 'TOP_LANE',
  'position': {'x': 981, 'y': 10441},
  'teamId': 100,
  'timestamp': 838583,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 0,
  'laneType': 'MID_LANE',
  'position': {'x': 5846, 'y': 6396},
  'teamId': 100,
  'timestamp': 1004048,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 8,
  'laneType': 'BOT_LANE',
  'position': {'x': 10504, 'y': 1029},
  'teamId': 100,
  'timestamp': 1096123,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [1, 4, 5],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 2,
  'laneType': 'MID_LANE',
  'position': {'x': 8955, 'y': 8510},
  'teamId': 200,
  'timestamp': 1235403,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [2]

laneType와 teamPosition이 같아야 하는데 서로 표기되어 있는 이름이 조금씩 다르다. 하지만 [:3]까지만 가져온다면 서로 중복되는 이름이 존재하는데 이를 이용해서 함수로 생성한다.  
TOP_LANE <-> TOP

In [32]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0] == lane[0]) and (tower_team == team):
        return 1
    else:
        return 0

In [16]:
tower_tmp = list(map(lambda x: (x['laneType'], x['teamId'], x['timestamp']), tower_log))

In [17]:
tower_tmp

[('TOP_LANE', 100, 838583),
 ('MID_LANE', 100, 1004048),
 ('BOT_LANE', 100, 1096123),
 ('MID_LANE', 200, 1235403),
 ('TOP_LANE', 200, 1406358),
 ('BOT_LANE', 100, 1415902),
 ('TOP_LANE', 100, 1495974),
 ('MID_LANE', 100, 1516556)]

In [19]:
def assist_calc(x): 
    try:
        return '.'.join(list(map(lambda y: str(y), x['assistingParticipantIds'])))
    except:
        return '-'

In [119]:
def raw_data():
    api_key = 'RGAPI-74b215a9-7629-4146-8b43-04ad3366536b'
    page = randint(1, 50)
    url = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page='+str(page)+'&api_key='+api_key
    res = requests.get(url).json()
    summonerName_lst = list(map(lambda x: x['summonerName'], res))
    tmp = get_lst(summonerName_lst)
    df = pd.DataFrame(tmp, columns = ['gameid', 'matchs', 'timelines'])
    tmp_df = df.copy()
    lst = []
    
    for i in range(len(tmp_df)):
        if tmp_df.iloc[i].matchs['info']['gameMode'] == 'CLASSIC':
            g_id = tmp_df.iloc[i].gameid
            match = tmp_df.iloc[i].matchs['info']
            timeline = tmp_df.iloc[i].timelines['info']
            for j in range(10):
                lst_tmp = []
                lst_tmp.append(str(g_id))
                lst_tmp.append(match['gameDuration'])
                lst_tmp.append(match['gameVersion'])
                lst_tmp.append(match['participants'][j]['summonerName'])
                lst_tmp.append(match['participants'][j]['participantId'])
                lst_tmp.append(match['participants'][j]['championName'])
                lst_tmp.append(match['participants'][j]['teamPosition'])
                lst_tmp.append(match['participants'][j]['teamId'])
                lst_tmp.append(match['participants'][j]['win'])
                lst_tmp.append(match['participants'][j]['kills'])
                lst_tmp.append(match['participants'][j]['deaths'])
                lst_tmp.append(match['participants'][j]['assists'])
                lst_tmp.append(match['participants'][j]['totalDamageDealtToChampions'])
                lst_tmp.append(match['participants'][j]['totalDamageTaken'])

                ban_blue = list(map(lambda x: str(x['championId']), match['teams'][0]['bans']))
                ban_red = list(map(lambda x: str(x['championId']), match['teams'][1]['bans']))
                ban_lst_tmp = list(set(ban_blue + ban_red))
                ban_lst = '|'.join(ban_lst_tmp)
                lst_tmp.append(ban_lst)
                kill_log = list(map(lambda x: list(filter(lambda y: y['type'] == 'CHAMPION_KILL', x['events'])), timeline['frames']))
                kill_log = [element for array in kill_log for element in array]
                k = list(map(lambda x: str(x['killerId']), kill_log))
                v = list(map(lambda x: str(x['victimId']), kill_log))
                a = list(map(assist_calc, kill_log))
                lst_tmp.append('|'.join(k))
                lst_tmp.append('|'.join(v))
                lst_tmp.append('|'.join(a))

                total_gold = list(map(lambda x: str(x['participantFrames'][str(j+1)]['totalGold']), timeline['frames']))
                lst_tmp.append('|'.join(total_gold))

                tower_log = list(map(lambda x: list(filter(lambda y: y['type'] == 'BUILDING_KILL', x['events'])), timeline['frames']))
                tower_log = [element for array in tower_log for element in array]
                try:
                    log_lane = tower_log[0]['laneType']
                    log_team = tower_log[0]['teamId']
                except:
                    log_lane = '-'
                    log_team = '-'
                ft = lane_processing(log_lane, log_team, lst_tmp[6], lst_tmp[7]) 
                #lst_tmp[6] = teamPosition, lst_tmp[7] = teamId
                lst_tmp.append(ft)

                tower_tmp = list(map(lambda x: (x['laneType'], x['teamId'], x['timestamp']), tower_log))
                try:
                    laneTower = list(filter(lambda x: (x[0][0] == lst_tmp[6][0]) & (x[1] == lst_tmp[7]), tower_tmp))[0] 
                    #여러 로그 중 처음 로그만 확인할 것이다.
                    lane_flag = 1
                    laneTowerTime = laneTower[-1] #timestamp
                except:
                    lane_flag = 0
                    laneTowerTime = 0

                lst_tmp.append(lane_flag)
                lst_tmp.append(laneTowerTime)

                try:
                    g_15 = timeline['frames'][14]['participantFrames'][str(j+1)]['totalGold']
                except:
                    g_15 = '-'
                lst_tmp.append(g_15)

                level_log = list(map(lambda x: list(filter(lambda y: y['type'] == 'LEVEL_UP' and y['level'] == 6, x['events'])), timeline['frames']))
                level_log = [element for array in level_log for element in array]
                try:
                    lv6_time = list(filter(lambda x: x['participantId'] == j+1, level_log))[0]['timestamp']
                except:
                    lv6_time = 0
                lst_tmp.append(lv6_time)

                lst.append(lst_tmp)

    col_lst=['gameId','gameDuration','gameVersion','summonerName',
             'participantId','championName', 'teamPosition', 'teamId','win','kills','deaths', 'assists',
             'damageDealt','damageTaken','bans','killerId','victimId','assistId', 
             'gold_5_35','firstLaneTower','laneTower','timeLaneTower','g15','lv6_time']

    df2 = pd.DataFrame(lst, columns = col_lst)

    db_open()
    df2.progress_apply(lambda x: db_insert(x), axis = 1)
    db_close()

    return

In [99]:
df2

,gameId,gameDuration,gameVersion,summonerName,participantId,championName,teamPosition,teamId,win,kills,...,bans,killerId,victimId,assistId,gold_5_35,firstLaneTower,laneTower,timeLaneTower,g15,lv6_time
0,KR_5959757493,1689,12.11.445.6412,olaolaolaola,1,Graves,TOP,100,False,0,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|563|860|1204|1599|1984|2267|2654|3072|...,1,1,838583,5062,341413
1,KR_5959757493,1689,12.11.445.6412,Sehee lu,2,Nidalee,JUNGLE,100,False,8,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|666|1013|1313|1815|2349|2721|3058|3709...,0,0,0,5657,384814
2,KR_5959757493,1689,12.11.445.6412,길고양이츄르뺏기,3,Yone,MIDDLE,100,False,3,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|584|850|1092|1298|1755|2059|2348|2760|...,0,1,1004048,4446,399084
3,KR_5959757493,1689,12.11.445.6412,Senaphine,4,Seraphine,BOTTOM,100,False,3,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|563|790|1018|1467|1667|2044|2452|2687|...,0,1,1096123,4393,507936
4,KR_5959757493,1689,12.11.445.6412,need a win,5,Senna,UTILITY,100,False,4,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|583|793|1085|1426|1625|1898|2160|2396|...,0,0,0,4256,526763
5,KR_5959757493,1689,12.11.445.6412,카시스 페델리안,6,Sett,MIDDLE,200,True,3,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|612|1004|1260|1991|2689|2916|3275|3468...,0,1,1235403,5616,313137
6,KR_5959757493,1689,12.11.445.6412,xoooS2,7,Taliyah,JUNGLE,200,True,6,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|606|1023|1378|1500|2098|2765|2987|3548...,0,0,0,5868,410678
7,KR_5959757493,1689,12.11.445.6412,생배낮듀혐오자,8,Akshan,TOP,200,True,12,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|542|941|1190|1543|1834|2506|2795|3186|...,0,1,1406358,6731,352744
8,KR_5959757493,1689,12.11.445.6412,여덟 조각 복근,9,Draven,BOTTOM,200,True,3,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|584|990|1316|1627|2552|2935|3336|3764|...,0,0,0,5382,444864
9,KR_5959757493,1689,12.11.445.6412,LSB DooTi,10,Renata,UTILITY,200,True,1,...,55|53|41|58|62|221|-1|7,6|2|3|9|7|2|8|2|5|6|8|5|7|2|8|2|5|8|2|2|3|4|9|...,3|10|9|2|3|6|2|6|10|3|2|10|1|9|1|10|8|3|7|10|1...,-|4.5|-|-|9|-|7|3|-|7.8|6.7|4|8|4.5|-|4.5|3.4|...,500|500|563|777|996|1150|1309|1483|1617|1862|2...,0,0,0,2909,672458


In [109]:
db_open()
query = """
    create table tower_info(gameId varchar(50), gameDuration number(20), gameVersion varchar(50), summonerName varchar(100),
    participantId varchar(20), championName varchar(50), teamPosition varchar(20), teamId varchar(10), 
    win varchar(50), kills number(10), deaths number(10), assists number(10), damageDealt number(10), damageTaken number(10), 
    bans varchar(500), killerId varchar(500), victimId varchar(500), assistId varchar(500), gold_5_35 varchar(500),
    firstLaneTower number(10), laneTower number(10), timeLaneTower number(10), g15 number(10), lv6_time number(10))
"""

sql_execute(query)
db_close()

db_open!
db_close!


In [107]:
db_open()
query = "drop table tower_info"
sql_execute(query)
db_close()

db_open!
db_close!


In [110]:
def db_insert(x):
    query = (
        f"insert into tower_info(gameId, gameDuration, gameVersion, summonerName, "
        f"participantId, championName, teamPosition, teamId, win, kills, deaths, assists, "
        f"damageDealt, damageTaken, bans, killerId, victimId, assistId, " 
        f"gold_5_35, firstLaneTower, laneTower, timeLaneTower, g15, lv6_time)"
        f"values(\'{x.gameId}\', {x.gameDuration}, \'{x.gameVersion}\', \'{x.summonerName}\', \'{x.participantId}\', "
        f"\'{x.championName}\', \'{x.teamPosition}\', \'{x.teamId}\', \'{x.win}\', {x.kills}, {x.deaths}, {x.assists}, "
        f"{x.damageDealt}, {x.damageTaken}, \'{x.bans}\', \'{x.killerId}\', \'{x.victimId}\', \'{x.assistId}\', "
        f"\'{x.gold_5_35}\', {x.firstLaneTower}, {x.laneTower}, {x.timeLaneTower}, {x.g15}, {x.lv6_time})"
    )
    sql_execute(query)
    return

In [111]:
db_open()
df2.progress_apply(lambda x: db_insert(x), axis = 1)
db_close()

db_open!


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 760.08it/s]

db_close!


In [117]:
from random import *

In [126]:
import time

In [ ]:
for i in range(100):
    raw_data()
    time.sleep(120)

db_open!


1it [00:00, 1001.51it/s]

db_close!


db_open!


100%|█████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 694.52it/s]

db_close!
